In [233]:
import sys
from surprise.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cosine, correlation
from sklearn.metrics import pairwise_distances
from collections import defaultdict
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import Dataset
from surprise import Reader

In [27]:
import os
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation
from scipy.stats import norm
%matplotlib inline
import seaborn as sns
sns.set(color_codes=True)
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score


In [3]:
userFeature = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
ratingFeature = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
genreFeature = ['genre']
moviesFeature = ['movie_id', 'title', 'release_date', 'video_release_date','imdb_url']
train_file ='u1.base'
test_file = 'u1.test'

In [4]:
def importFile(fileName,feature,sep,crange):
    if sep == None:
        sep = '|'
    if crange == None:
        df = pd.read_csv(fileName, sep=sep,names=feature,encoding='latin-1')
    else:
        df = pd.read_csv(fileName, sep=sep,names=feature,usecols=range(crange),encoding='latin-1')
    return df

In [124]:
feature = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
df = pd.read_csv('~/Dev/datasets/ml-100k/u.data', 
                 sep='\t',
                 names=feature,
                 encoding='latin-1')

In [131]:
df.head(10)

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
5,298,474,4,884182806
6,115,265,2,881171488
7,253,465,5,891628467
8,305,451,3,886324817
9,6,86,3,883603013


In [143]:
user2 = df.loc[df['user_id'] == 2]
user2.to_csv('user2.csv')

In [144]:
#ratting_matriix_clustered.loc[ratting_matriix_clustered['cluster']
len(df.loc[df['user_id'] == 2])

62

In [157]:
user1 = df.loc[df['user_id'] == 1]
user1.to_csv('user1.csv')
len(df.loc[df['user_id'] == 1])

272

In [195]:
pivotRatingDf = df.pivot_table(index=['user_id'],columns=['movie_id'],values='rating').reset_index(drop=True)
pivotRatingDf.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [196]:
pivotRatingDf.fillna(0,inplace=True)
pivotRatingDf.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [197]:
pivotRatingDf.shape

(943, 1682)

In [198]:
X = pivotRatingDf[pivotRatingDf.columns[:]] # ไม่เอา 1 คอลัมแรก และ 2 คอลัมท้าย
X.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [199]:
pivotRatingDf.to_csv('ratingMatrix_u_i.csv');

### Clustering User

In [200]:
range_n_clusters = [2, 3, 4, 5, 6]

for n_clusters in range_n_clusters:
    clusterer = KMeans(n_clusters=n_clusters, random_state=2, n_init=10)
    cluster_labels = clusterer.fit_predict(X)
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

For n_clusters = 2 The average silhouette_score is : 0.23384760967073315
For n_clusters = 3 The average silhouette_score is : 0.12897114892753064
For n_clusters = 4 The average silhouette_score is : 0.12980866054642248
For n_clusters = 5 The average silhouette_score is : 0.029151467002173945
For n_clusters = 6 The average silhouette_score is : 0.018684746836156504


In [201]:
n_clusters = 2
kmeans = KMeans(n_clusters=n_clusters, n_init=10)
kmeans.fit(X)
labels = kmeans.labels_
print(len(labels))
#print(labels)

943


In [202]:
pd.DataFrame(labels).head(5)

,0
0,0
1,1
2,1
3,1
4,0


In [203]:
ratting_matriix_clustered = pivotRatingDf
ratting_matriix_clustered['cluster'] = labels
ratting_matriix_clustered.head(10)

movie_id,1,2,3,4,5,6,7,8,9,10,...,1674,1675,1676,1677,1678,1679,1680,1681,1682,cluster
0,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5,4.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
6,0.0,0.0,0.0,5.0,0.0,0.0,5.0,5.0,5.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
7,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
8,0.0,0.0,0.0,0.0,0.0,5.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
9,4.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [173]:
#ratting_matriix_clustered[ratting_matriix_clustered.columns[:-1]] #ignore last column

In [208]:
cluster0 = ratting_matriix_clustered.loc[ratting_matriix_clustered['cluster'] == 0]
cluster1 = ratting_matriix_clustered.loc[ratting_matriix_clustered['cluster'] == 1]
cluster0.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1674,1675,1676,1677,1678,1679,1680,1681,1682,cluster
0,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5,4.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
6,0.0,0.0,0.0,5.0,0.0,0.0,5.0,5.0,5.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9,4.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [209]:
cluster1.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1674,1675,1676,1677,1678,1679,1680,1681,1682,cluster
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
7,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
8,0.0,0.0,0.0,0.0,0.0,5.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [211]:
cluster0 = cluster0[cluster0.columns[:-1]] #ignore last column
#cluster0.sample(5)
cluster0.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,5.0,0.0,0.0,5.0,5.0,5.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,4.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [210]:
cluster1 = cluster1[cluster1.columns[:-1]] #ignore last column
cluster1.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,5.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [212]:
cluster0.shape

(296, 1682)

In [213]:
#for index, rows in cluster0.iterrows():
#    print(index)

Convert cluster 0 from matrix to normal table

In [236]:
ratting_list = []
for index, rows in cluster0.iterrows():
    #for i in range(1,1683):
         #print(i)
    for i,e in enumerate(rows):
        if e != 0:
            d = {'user_id': index+1, 'movie_id':i+1,'ratting':e}
            ratting_list.append(d)
        #print(i+1,e)

In [237]:
cluster0_df = pd.DataFrame(ratting_list)
cluster0_df.head(5)

,movie_id,ratting,user_id
0,1,5.0,1
1,2,3.0,1
2,3,4.0,1
3,4,3.0,1
4,5,3.0,1


In [238]:
len(cluster0_df.loc[custer0_df['user_id'] == 1])

272

In [239]:
user1 = cluster0_df.loc[cluster0_df['user_id'] == 1]
user1.to_csv('cluster0_user1.csv')

Convert cluster 1 from matrix to normal table

In [218]:
cluster1.shape

(647, 1682)

In [244]:
ratting_list = []
for index, rows in cluster1.iterrows():
    for i,e in enumerate(rows):
        if e != 0:
            d = {'user_id': index+1, 'movie_id':i+1,'ratting':e}
            ratting_list.append(d)

In [245]:
cluster1_df = pd.DataFrame(ratting_list)
cluster1_df.head(5)

,movie_id,ratting,user_id
0,1,4.0,2
1,10,2.0,2
2,13,4.0,2
3,14,4.0,2
4,19,3.0,2


In [246]:
user2 = cluster1_df.loc[cluster1_df['user_id'] == 2]
user2.to_csv('cluster1_user2.csv')
len(cluster1_df.loc[cluster1_df['user_id'] == 2])

62

In [247]:
user1.tail(5)

,movie_id,ratting,user_id
267,268,5.0,1
268,269,5.0,1
269,270,5.0,1
270,271,2.0,1
271,272,3.0,1


### Evaluation

All user

In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'movie_id', 'rating']], reader)
algorithms = SVDpp()

#trainset = data.build_full_trainset()
#result = algorithms.fit(trainset)

cross_validate(algorithms, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Cluster 0 User

In [255]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(cluster0_df[['user_id', 'movie_id', 'ratting']], reader)
algorithms = SVDpp()

cross_validate(algorithms, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9087  0.9011  0.8983  0.9002  0.8862  0.8989  0.0073  
MAE (testset)     0.7096  0.7086  0.7040  0.7049  0.6953  0.7045  0.0051  
Fit time          136.14  131.71  132.01  133.31  149.86  136.61  6.81    
Test time         2.55    2.59    2.69    2.60    2.76    2.64    0.08    


{'test_rmse': array([0.90874601, 0.90109294, 0.89828862, 0.90023005, 0.88616617]),
 'test_mae': array([0.70959003, 0.70861064, 0.70398186, 0.70494752, 0.69527022]),
 'fit_time': (136.13879704475403,
  131.7093141078949,
  132.00791001319885,
  133.3121259212494,
  149.8624038696289),
 'test_time': (2.5490520000457764,
  2.5886902809143066,
  2.6943347454071045,
  2.600008964538574,
  2.7621829509735107)}

Cluster 1 User

In [254]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(cluster1_df[['user_id', 'movie_id', 'ratting']], reader)
algorithms = SVDpp()

cross_validate(algorithms, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9943  0.9898  0.9871  0.9855  0.9836  0.9880  0.0037  
MAE (testset)     0.7836  0.7824  0.7794  0.7780  0.7779  0.7803  0.0023  
Fit time          24.79   24.77   26.34   24.12   22.74   24.55   1.16    
Test time         0.57    0.96    1.16    0.47    0.49    0.73    0.28    


{'test_rmse': array([0.99429193, 0.98976112, 0.98707334, 0.98548845, 0.98363197]),
 'test_mae': array([0.78355365, 0.78244015, 0.77941293, 0.77797299, 0.77792755]),
 'fit_time': (24.78566813468933,
  24.774256944656372,
  26.3371319770813,
  24.121885776519775,
  22.739444971084595),
 'test_time': (0.5713930130004883,
  0.9602029323577881,
  1.1563332080841064,
  0.47095179557800293,
  0.492419958114624)}